In [ ]:
# Install all the required dependencies for the project
!pip install pytorch-lightning==1.6.5
!pip install spacy==2.2.4
!python -m spacy download en_core_web_md
!pip install scikit-learn==1.0.2
!pip install matplotlib # to render Confusion Matrix
!pip install seaborn # to render Confusion Matrix

Import all the necessary libraries we need throughout the project.

In [1]:
# Import all the relevant libraries
import spacy
import en_core_web_md
import csv
import itertools

import pytorch_lightning as pl
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, random_split
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import LabelEncoder
import torchmetrics
from torch.utils.data import DataLoader, TensorDataset

In [2]:
loaded_spacy_model = en_core_web_md.load()

In [14]:
sentences = [
  "test1 test2 test3",
  "test1 test2",
  "test1"
]
classes = [1, 2, 3]

# [
#   [(300), (300), (300)],
#   [(300), (300)],
#   [(300)]
# ]
sentences_vectors = []
for sentence in sentences:
  spacy_doc = loaded_spacy_model.make_doc(sentence)
  word_vectors = [token.vector for token in spacy_doc]
  sentences_vectors.append(word_vectors)

# [
#   tensor((300)),
#   tensor((300)),
#   tensor((300))
# ]
word_vectors_tensor = [torch.Tensor(sentence_vectors) for sentence_vectors in sentences_vectors]

# tensor(
#   tensor((300), (300), (300)),
#   tensor((300), (300), (300)),
#   tensor((300), (300), (300)), --> tensor (900)
# )
# (max_seq_len, batch_size, word_vector_dim)
batch = pad_sequence(word_vectors_tensor)


# hyperparameters
lstm_input_size = 300
lstm_hidden_size = 128
lstm_num_layers = 1
classes_number = 3

# models
lstm = torch.nn.LSTM(
  input_size = lstm_input_size,
  hidden_size = lstm_hidden_size,
  num_layers = lstm_num_layers,
)
linear = torch.nn.Linear(lstm_hidden_size, classes_number)

# word word word ..... -> class

# forward step
h0 = torch.ones(lstm_num_layers, batch.shape[1], lstm_hidden_size)
c0 = torch.ones(lstm_num_layers, batch.shape[1], lstm_hidden_size)
output_lstm, (final_hidden_state, final_cell_state) = lstm(batch, (h0, c0))
output_linear = linear(output_lstm)
output_linear

tensor([[[ 0.1406,  0.0046,  0.0920],
         [ 0.1406,  0.0046,  0.0920],
         [ 0.1406,  0.0046,  0.0920]],

        [[ 0.0598,  0.0111, -0.0198],
         [ 0.0598,  0.0111, -0.0198],
         [ 0.0711, -0.0101, -0.0199]],

        [[ 0.0173,  0.0219, -0.0438],
         [ 0.0414, -0.0108, -0.0302],
         [ 0.0351, -0.0206, -0.0155]]], grad_fn=<ViewBackward0>)